In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import gc
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, log_loss
from imblearn.under_sampling import RandomUnderSampler
import gc
from keras.models import Model
from keras.layers import Dense, Conv1D, Dropout, Input, GlobalAveragePooling1D, MaxPool1D, Flatten, LSTM

In [ ]:
questions = pd.read_csv('../input/train.csv')
questions.info()

In [ ]:
os.listdir('../input/embeddings/')

In [ ]:
max_len = 200

In [ ]:
embeddings = dict()
# file = open('../input/embeddings/glove.840B.300d/glove.840B.300d.txt', encoding='UTF-8')
file = open('../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec', 
            encoding='UTF-8')
for line in file:
    values = line.split(" ")
    word, coefs  = values[0], np.array(values[1:], dtype='float32')
    embeddings[word] = coefs

In [ ]:
tkn = Tokenizer()
tkn.fit_on_texts(questions.question_text)
vocab_size = len(tkn.word_index) + 1
enc_sentence = tkn.texts_to_sequences(questions.question_text)
enc_sentence = pad_sequences(enc_sentence, max_len)

In [ ]:
embed_matrix = np.zeros((vocab_size, 300))
for word, i in tkn.word_index.items():
    vec = embeddings.get(word)
    if(vec is not None):
        embed_matrix[i] = vec

In [ ]:
sent_vecs = np.zeros((len(enc_sentence), 300))
for i in range(len(enc_sentence)):
    word_vec = np.array([embed_matrix[w] for w in enc_sentence[i]])
    sent_vec = np.sum(word_vec, axis=0)
    sent_vec = sent_vec/sqrt((sent_vec**2).sum())
    sent_vec = np.nan_to_num(sent_vec)
    # sent_vec = sent_vec.reshape((300, 1))
    sent_vecs[i] = sent_vec
sent_vecs.shape

In [ ]:
gc.collect()

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_sample(sent_vecs, questions.target)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, 
                                                    test_size = 0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(sent_vecs, questions.target, 
#                                                    test_size = 0.2, random_state=42)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_test.reshape([X_test.shape[0], X_test.shape[1]]), y_test, 
                                                    test_size = 0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.kernel_approximation import Nystroem
from sklearn import svm

clf = svm.LinearSVC(C=21.5443, tol=0.0001)
feature_map_nystroem = Nystroem(gamma=.0013, random_state=1234, n_components=500)

# data_transformed = feature_map_nystroem.fit_transform(X_train)

clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_val)
f1_score(y_pred=preds, y_true=y_val), log_loss(y_pred=preds, y_true=y_val)

In [ ]:
questions_test = pd.read_csv('../input/test.csv')
questions_test.info()

In [ ]:
enc_sentence_test = tkn.texts_to_sequences(questions_test.question_text)
enc_sentence_test = pad_sequences(enc_sentence_test, max_len)
len(enc_sentence_test)

In [ ]:
sent_vecs_test = np.zeros((len(enc_sentence_test), 300))
for i in range(len(enc_sentence_test)):
    word_vec = np.array([embed_matrix[w] for w in enc_sentence_test[i]])
    sent_vec = np.sum(word_vec, axis=0)
    sent_vec = sent_vec/sqrt((sent_vec**2).sum())
    sent_vec = np.nan_to_num(sent_vec)
    # sent_vec = sent_vec.reshape((300, 1))
    sent_vecs_test[i] = sent_vec
sent_vecs_test.shape

In [ ]:
preds = clf.predict(sent_vecs_test)
submission = pd.DataFrame({'qid': questions_test.qid, 'prediction':preds})
submission.info()
submission.to_csv('./submission.csv', index=False)